# Pipelines

The Pipeline class allows us to combine multiple optimization processes into a single process. For example, we could do a coarse grid search to obtain an initial signal, then shrink the bounds to enclose the signal and sample with higher resolution, then finally fit all observed data to a model which predicts the location of the maximum. First, we define a cost function to optimize:

In [1]:
import numpy as np
def cost(state, params):
    return -np.exp(-(state['X']-params['x0'])**2)*np.exp(-(state['Y']-params['y0'])**2)

params = {'sigma_x': 0.3, 'sigma_y': 0.8, 'x0': 0.3, 'y0': 0.6, 'noise':0.0}

The cost function should have two arguments. The "state" is a dictionary containing the independent variables, and the "params" is a dictionary containing parameters defining the cost evaluation - in this case, 'x0' and 'y0' define the center position of a multivariate Gaussian. The pipeline will minimize the cost function, so make sure you set the sign of the return value appropriately.

Now let's define our initial state and the bounds:

In [2]:
state = {'X': 0, 'Y': 2}
bounds = {'X': (-2,2), 'Y': (-4, 4)}

Now we'll instantiate the pipeline object:

In [ ]:
from emergent.pipeline import Pipeline
pipe = Pipeline(cost, params, state, bounds)


We can add blocks to the Pipeline to define the optimization process we want to apply. Let's start with a simple 10x10 grid search:

In [ ]:
from emergent.pipeline import GridSearch
pipe.add(GridSearch({'Steps': 10}))

Once we've added all of the blocks we want, we can run the pipeline:

In [ ]:
points, costs = pipe.run()


This basic grid search came within 1% of the true minimum with only 102 function evaluations (once when initializing the pipeline, 100 times during optimization, and once more to check the final result). However, as the dimensionality of the parameter space grows, the number of iterations required grows exponentially! A coarse 10-point-per-axis grid search in six dimensions would take a million iterations to complete!

For optimization in higher dimensions, a number of other algorithms are included. I will demonstrate these in the following examples.

# Gradient-based methods
The grid search is inefficient because all of the sampled points are chosen before starting the optimization. In many cases, the minimum can be found more efficiently by choosing each subsequent point based on knowledge of the parameter space. The simplest way to do this is to measure the local gradient, take a step downhill, and repeat. This is implemented by the GradientDescent algorithm - let's try it out. We'll rebuild the pipeline from scratch, but make sure you've run the above blocks in the current kernel to define the cost function, initial state, and bounds.

In [ ]:
from emergent.pipeline import Pipeline, GradientDescent

pipe = Pipeline(cost, params, state, bounds)
pipe.add(GradientDescent())
points, costs = pipe.run()

The GradientDescent comes about as close to the local minimum in half the experimental iterations of the GridSearch method, and the relative performance should increase in higher dimensional parameter spaces.

The L-BFGS-B algorithm is another option, tending to perform excellently in low dimensional spaces:

In [ ]:
from emergent.pipeline import Pipeline, LBFGSB

pipe = Pipeline(cost, params, state, bounds)
pipe.add(LBFGSB())
points, costs = pipe.run()

Lastly, the Adam optimizer incorporates momentum to more efficiently navigate valleys and saddle points. 

In [ ]:
from emergent.pipeline import Pipeline, Adam

pipe = Pipeline(cost, params, state, bounds)
pipe.add(Adam())
points, costs = pipe.run()

# Stochastic methods
Gradient-based methods can be inefficient for several reasons:
* High-dimensional parameter spaces often have many local minima where the optimizer will get trapped.
* Measurement noise will degrade the gradient accuracy, which can cause the optimizer to move in the wrong direction or get stuck in regions where the noise is larger than the local gradient
* Gradient computations in N dimensions require 2N function evaluations, which can take a while.

An alternative is to use semi-random methods where samples are generated based on knowledge of other samples. For example, the DifferentialEvolution algorithm creates a population of points and carries out generations of "breeding" to select for regions of parameter space where the cost function is low:

In [ ]:
from emergent.pipeline import Pipeline, DifferentialEvolution

pipe = Pipeline(cost, params, state, bounds)
pipe.add(DifferentialEvolution())
points, costs = pipe.run()

The ParticleSwarm optimizer similarly initializes a random set of particles which move through space to seek both their best known position and the best known position of the swarm:

In [ ]:
from emergent.pipeline import Pipeline, ParticleSwarm

pipe = Pipeline(cost, params, state, bounds)
pipe.add(ParticleSwarm())
points, costs = pipe.run()

# Model-based methods
Online optimization is a powerful strategy, especially for expensive, black-box cost functions. The basic idea is to form a closed loop consisting of:
1. Fit a parameter surface to observed data.
2. Numerically optimize the modeled surface to generate the next point.
3. Repeat until convergence.

Models are implemented as subpipelines which consume real data from the primary pipeline to train, then run through a set of user-defined blocks to numerically optimize the modeled surface:

In [3]:
from emergent.pipeline import *     

pipe = Pipeline(cost, params, state, bounds)
pipe.add(GridSearch({'Steps': 5}))      ## acquire initial data with a coarse grid search
model = pipe.add(GaussianModel())       # attach a model subpipeline
grad = model.add(ParticleSwarm())            # run gradient descent to optimize the modeled surface
    
points, costs = pipe.run()      # run through all attached blocks, including the model subpipeline

INFO:root:Optimization complete!
INFO:root:Time: 0s
INFO:root:Evaluations: 28
INFO:root:Initial cost: -0.128735
INFO:root:Final cost: -0.943252
INFO:root:Improvement: 632.7%


After training a model, the surface can be visualized by calling Model.plot(axis), where "axis" is an integer corresponding to the order of the state dict.

In [9]:
%gui qt     
# ^ specify GUI backend for Jupyter compatibility
model.plot(1)

(<pyqtgraph.graphicsWindows.GraphicsWindow at 0x1a77ff08438>,
 <pyqtgraph.SignalProxy.SignalProxy at 0x1a77ff29708>)

In [ ]:
model.pipeline.unnormalize(model.points)